# Import and functions

In [19]:
import numpy as np
from qutip import *
import os 
from itertools import product


In [29]:
def prob_distr(rho, projs, marg=[]): 
    # Number of parties
    num_p = len(projs) 
    # Number of inputs for each party
    inputs = [len(projs[i]) for i in range(num_p)]
    # Number of outputs for each party
    outputs_compl = [projs[i][0][0].shape[0] for i in range(num_p)]
    outputs = [len(projs[i][0]) for i in range(num_p)]
    
    # Remove inputs and ouputs of the marginalized parties
    if len(marg) != 0:
        # Check that the marginalized party exists
        if any(m > num_p-1 for m in marg):
            raise ValueError(f"The indices of the marginalized parties should be smaller than {num_p-1}")
        for m in sorted(marg, reverse=True):
            inputs.pop(m)
            outputs.pop(m)
            
    # Initialize the distribution
    distr = np.empty(shape=(*outputs, *inputs), dtype=complex)
    input_idx = [list(range(i)) for i in inputs]
    output_idx = [list(range(o)) for o in outputs]

    # Iterate over all combinations of inputs (x,y,z...) and outputs, creating indices 'ind' of the form (a,b,c,...x,y,z...)
    for i in product(*input_idx):
        for o in product(*output_idx):
            ops = []
            for p in range(num_p):
                # If the party has been marginalized, insert an identity, otherwise the corresponding projector
                if p in marg:
                    ops.append(qeye(outputs_compl[p]))
                else:
                    count = sum(map(lambda x : x<p, marg)) # Number of parties previously marginalized
                    ops.append(projs[p][i[p-count]][o[p-count]])
            
            meas_op = tensor(*ops) # Tensor product of all the projectors indicated by 'ind'
            np_meas_op = array(meas_op.full(), dtype=complex)
            np_rho = array(rho.full(), dtype=complex)
            ind = o + i
            distr[ind] = np.trace(np_meas_op @ np_rho, dtype= complex)
            
    return np.real(distr)

# Compute the correlator <A_x B_y C_z> from the corresponding distribution p(a,b,c|x,y,z)
def prob_tocorr3D_BSM(distr, x, y, z):
    corr = 0
    for a,b,c in list(product([0,1], ['00','01','10','11'], [0,1])):
        corr += (-1)**(a+int(b[y])+c) * distr[a,int(b, 2),c,x,0,z]
    return corr

def J14(distr):
    J = 0
    for x,z in product([0,1], repeat=2):
        J += (-1)**(x+z)*prob_tocorr3D_BSM(distr, x, 1, z)
    return J/4
    
def I14(distr):
    I = 0
    for x,z in product([0,1], repeat=2):
        I += prob_tocorr3D_BSM(distr, x, 0, z)
    return I/4

# Branciard-Rosset-Gisin-Pironio bilocal inequality
def BRGP(distr):
    return np.sqrt(abs(I14(distr))) + np.sqrt(abs(J14(distr)))

# Return the projectors of the operator 'op'
def projs(op):
    return [op.eigenstates()[1][i].proj() for i in range(op.shape[0])]


In [24]:
# Experimental counts
with np.load('Bilocality_sperimentale/exp_counts.npz') as data:
        exp_counts = data['exp_counts']

# Monte Carlo (exp. probability distributions)

In [25]:
N_estr = 100

savez_dict = dict()
for c,b in enumerate(exp_counts):
    P_g_estr = []
    I_estr = []
    for i in np.arange(N_estr):
           
        counts_poiss = np.random.poisson(b)
        distr = counts_poiss#/np.sum(counts_poiss, axis=(0,1,2))
        distr_n = counts_poiss/np.sum(counts_poiss, axis=(0,1,2))
        P_g_estr.append(distr)
        I_estr.append(BRGP(distr_n))
    savez_dict[f'distr_exp_{c}'] = P_g_estr
    savez_dict[f'BRGP_exp_{c}'] = I_estr

    
      
#np.savez("exp_data/exp_counts.npz", **savez_dict)

# Theoretical model distributions

In [30]:
# Measurements
A0 = (sigmaz() + sigmax())/np.sqrt(2)
A1 = (sigmaz() - sigmax())/np.sqrt(2)

B00 = bell_state('00')*bell_state('00').dag()
B01 = bell_state('01')*bell_state('01').dag()
B10 = bell_state('10')*bell_state('10').dag()
B11 = bell_state('11')*bell_state('11').dag()

C0 = (sigmaz() + sigmax())/np.sqrt(2)
C1 = (sigmaz() - sigmax())/np.sqrt(2)


# State
vis = 0.89
col = 0.33
psi1 = bell_state('11')
psi2 = psi1.copy()
rho1 = psi1*psi1.dag()
rho2 = psi2*psi2.dag()

ket01 = tensor(basis(2, 0), basis(2, 1))
ket10 = tensor(basis(2, 1), basis(2, 0))
rho01_1 = ket01*ket01.dag()
rho10_1 = ket10*ket10.dag()
rho01_2 = rho01_1.copy()
rho10_2 = rho10_1.copy()
psi1_vis = vis*rho1 + (1-vis)*(col/2*(rho01_1 + rho10_1) + (1-col)*qeye(rho1.dims[0])/rho1.shape[0])
psi2_vis = vis*rho2 + (1-vis)*(col/2*(rho01_2 + rho10_2) + (1-col)*qeye(rho2.dims[0])/rho2.shape[0])
rho_tot = tensor(psi1_vis, psi2_vis)


N_p = 30

p_ind =np.linspace(0, 1, N_p)

distr_tot = []
BRGP_tot = []

for cp, p in enumerate(p_ind):
    
    B00_p = (1+p)/2*bell_state('00')*bell_state('00').dag() + (1-p)/2*bell_state('01')*bell_state('01').dag()
    B01_p = (1+p)/2*bell_state('01')*bell_state('01').dag() + (1-p)/2*bell_state('00')*bell_state('00').dag()
    B10_p = (1+p)/2*bell_state('10')*bell_state('10').dag() + (1-p)/2*bell_state('11')*bell_state('11').dag()
    B11_p = (1+p)/2*bell_state('11')*bell_state('11').dag() + (1-p)/2*bell_state('10')*bell_state('10').dag()
    projs_p = [[projs(A0),projs(A1)], [[B00_p,B01_p,B10_p,B11_p]], [projs(C0),projs(C1)]]

    distr_tot.append(prob_distr(rho_tot, projs_p))
    distr_p = prob_distr(rho_tot, projs_p)
    BRGP_tot.append(BRGP(distr_p))

#savez_dict = {}
#savez_dict[f'distr_teo'] = distr_tot
#savez_dict[f'BRGP_teo'] = BRGP_tot    
#np.savez("exp_data/theo_model.npz", **savez_dict)